<h2>Desafio 2 - Machine Learning


<h3>Grupo 7</h3>
<ul>
    <li>Ignacio Mendieta</li>
    <li>Laura Jazmín Chao</li>
    <li>Juan Nicolás Capistrano</li>
    <li>Betiana Srur</li>
    <li>Marecelo Carrizo</li>
    
</ul>
    

**Desafio 2** : Aplicar las tecnicas de Machine Learning para calcular el precio por metro cuadrado de las propiedades.

In [15]:
import numpy as np
import pandas as pd
import re

# Libreria Grafica.
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

# Visualización
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import NumeralTickFormatter # Para imprimir los %
from bokeh.tile_providers import CARTODBPOSITRON, get_provider, Vendors 
from bokeh.resources import INLINE

# Libreria para analisis ML
from sklearn import datasets, linear_model

pd.set_option('display.max_columns', 100) # Para mostrar todas las columnas
pd.set_option('display.max_rows', 100) # Para mostrar todas las filas

In [16]:
# Definimos la ruta de la información.
data_propiedades = "Data/properati.csv"

# Leemos los datos del archivo
data = pd.read_csv(data_propiedades, sep=",", encoding="UTF-8")

# Chequeamos que los datos se hayan importado correctamente
data.sample(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
119868,119868,sell,apartment,Palermo,|Argentina|Capital Federal|Palermo|,Argentina,Capital Federal,3430234.0,"-34.588333,-58.419145",-34.588333,-58.419145,208000.0,USD,3670056.0,208000.0,NaN,68.0,NaN,3058.823529,NaN,2.0,NaN,http://www.properati.com.ar/1ch7n_venta_depart...,Corredor Responsable: Maria Cristina Barrionue...,VENTA DEPTO. 2 AMB. PALERMO - APTO CREDITO!,https://thumbs4.properati.com/3/MBNfoEaT8oocfR...
90104,90104,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,39.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1b9nj_venta_depart...,Sergio Villella Inmobiliaria presenta departam...,Av Francia 700 - Departamentos de un dormitori...,https://thumbs4.properati.com/4/Awfaf_n63HlB7b...
65469,65469,sell,apartment,Villa Ortuzar,|Argentina|Capital Federal|Villa Ortuzar|,Argentina,Capital Federal,3427387.0,NaN,NaN,NaN,119000.0,USD,2099695.5,119000.0,44.0,41.0,2704.545455,2902.439024,NaN,NaN,1000.0,http://www.properati.com.ar/1acdu_venta_depart...,"A ESTRENAR, APTO PROFESIONAL!! HERMOSO DEPARTA...",EXCELENTE UBICACION- EDIFICIO NUEVO APTO PROFE...,https://thumbs4.properati.com/6/Fj-lp-zNAGLT7q...
18744,18744,sell,house,Martínez,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3430813.0,NaN,NaN,NaN,395000.0,USD,6969577.5,395000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/177rt_venta_casa_m...,"EXCELENTE CHALET A METROS DE DARDO ROCHA, DESA...","Casa con Cochera en Monteagudo 800, San Isidro...",https://thumbs4.properati.com/3/d2l3T0fyanNgDu...
24643,24643,sell,house,Tres Arroyos,|Argentina|Buenos Aires Interior|Tres Arroyos|,Argentina,Buenos Aires Interior,3833857.0,"-38.373899,-60.2682316",-38.373899,-60.268232,80000.0,USD,1411560.0,80000.0,200.0,90.0,400.000000,888.888889,NaN,7.0,NaN,http://www.properati.com.ar/17osj_venta_casa_t...,"Casa en excelente ubicacion, barrio residencia...",CASA PLENO RESIDENCIAL,https://thumbs4.properati.com/9/lIGhq5C521uKVM...


In [17]:
# Chequeamos cantidad de registros y cantidad de columnas
data.shape

(121220, 26)

<h2>Selección de los datos</h2>

En base a los análisis previos realizados sobre el Dataset, tomaremos los datos de CABA únicamente para realizar un modelo de machine learning. 

In [18]:
# Creamos una máscara y la aplicamos al dataframe anterior para traer los registros que necesitamos
data_caba_mask = data.state_name == 'Capital Federal' 
data_caba = data.loc[data_caba_mask, :]

# Chequeamos cómo quedaron los datos
data_caba.head(3)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...


In [19]:
data_caba.shape

(32316, 26)

In [20]:
data_caba.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

<a id="section_dataset_nulos"></a> 
<h3>Cálculo de cantidad de nulos</h3>

[volver a TOC](#section_toc)

In [21]:
# # Calculamos cantidad de nulos por campo
# cant_nulos_por_campo = data_caba.isnull().sum()
# cant_nulos_por_campo

In [22]:
# # Calculamos porcentaje de nulos por campo
# cant_registros = data_caba.shape[0]
# print(f'Cantidad Original de registros: {cant_registros}\n')
# #len(data.index) es lo mismo

# porc_nulos_por_campo = np.round((100 * cant_nulos_por_campo / cant_registros),2)
# print(porc_nulos_por_campo)

<h4> Distribución de <strong>property_type</strong>

In [23]:
# Contamos cantidad de valores de cada elemento en property_type y operation para ver distribución
property_type_count = data_caba.property_type.value_counts()
print(f'property_type: \n{property_type_count}')

property_type: 
apartment    27037
house         2087
PH            1828
store         1364
Name: property_type, dtype: int64


In [24]:
# # Convertimos el count anterior en las categorías del gráfico
# categories = np.array(property_type_count.index)

# cord_x = data_caba.property_type.value_counts() / data_caba.shape[0]
# palette = ['#440154', '#29788E', '#22A784', '#FDE724']

# p = figure(x_range=categories, plot_width=500, plot_height=200)
# p.vbar(x=categories, top=cord_x, width=0.6,
#        color=palette)

# p.yaxis.formatter = NumeralTickFormatter(format='0 %')
# output_notebook(resources=INLINE)
# show(p)

<h4>Distribución de <strong>currency</strong></h4>

In [25]:
data_caba.currency.value_counts()

USD    27826
ARS     1190
PEN        2
Name: currency, dtype: int64

<h4>Distribución de <strong>barrio</strong></h4>

In [26]:
place_name_count = data_caba.place_name.value_counts()
# place_name_count.sort_index()
place_name_count

Belgrano                2992
Palermo                 2885
Caballito               2273
Villa Urquiza           1629
Recoleta                1547
Flores                  1354
Villa Crespo            1331
Capital Federal         1297
San Telmo               1216
Almagro                 1165
Barrio Norte            1140
Boedo                    890
Nuñez                    778
Balvanera                667
Puerto Madero            647
San Cristobal            614
Saavedra                 542
Monserrat                538
Palermo Hollywood        532
Floresta                 516
Villa del Parque         488
Villa Luro               432
Barracas                 425
Villa Devoto             420
Mataderos                418
Liniers                  408
Palermo Soho             394
Colegiales               352
Parque Patricios         322
Congreso                 304
Coghlan                  259
Retiro                   248
Palermo Chico            232
Chacarita                227
Centro / Micro

<h2>Limpieza de Datos</h2>

<a id="section_dataset_drop"></a> 
<h3> Drop de columnas innecesarias </h3>

[volver a TOC](#section_toc)

In [27]:
# Creamos la lista de columnas a filtrar
drop_columns = ['Unnamed: 0', 'operation', 'place_with_parent_names', 'country_name', 'state_name', 'geonames_id', 'lat-lon','lat', 'lon',
               'expenses','properati_url','title', 'image_thumbnail']

data_caba_clean = data_caba.drop(drop_columns, axis=1)
data_caba_clean.head(5)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
0,PH,Mataderos,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,..."
2,apartment,Mataderos,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...
3,PH,Liniers,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...
7,apartment,Belgrano,138000.0,USD,2434941.0,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...
8,apartment,Belgrano,195000.0,USD,3440677.5,195000.0,65.0,60.0,3000.000000,3250.000000,NaN,NaN,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...



<h3> Limpiamos <strong>currency</strong> </h3>

[volver a TOC](#section_toc)

In [28]:
# Creamos una mask para traer los valores en monedas que no son USD ni ARS.
currency_OTHER_CURRENCY_mask = (data_caba_clean.currency == 'PEN') | (data_caba_clean.currency == 'UYU')
# Observamos esos registros
data_caba_clean.loc[currency_OTHER_CURRENCY_mask, :]

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
50387,apartment,Capital Federal,380000.0,PEN,2066864.90,117139.33,NaN,NaN,NaN,NaN,NaN,NaN,Ref.: 55:PREVENTA ::: OPORTUNIDAD ::: Dúplex a...
50388,apartment,Capital Federal,950000.0,PEN,5167162.35,292848.33,NaN,NaN,NaN,NaN,NaN,NaN,Ref.: 56Duplex en venta de 1 dormitorio con ap...


In [29]:
# Hacemos drop de esos registros y chequeamos la diferencia entre cantidad original y actual
cant_registros = data_caba_clean.shape[0]
data_caba_clean.drop(data_caba_clean.loc[currency_OTHER_CURRENCY_mask, :].index, inplace = True)
print(f'Cantidad de registros original - registro actuales: {cant_registros - data_caba_clean.shape[0]}')

Cantidad de registros original - registro actuales: 2


<h3> Limpiamos <strong>place_name</strong> </h3>

[volver a TOC](#section_toc)

In [30]:
# Dropeamos los de Capital Federal
capi_mask = data_caba_clean.place_name == 'Capital Federal'
data_caba_clean.drop(data_caba_clean.loc[capi_mask, :].index, inplace=True)

# Chequeamos cómo quedaron los datos
data_caba_clean.shape

(31019, 13)

<h3> Volvemos a calcular cantidad de nulos luego de eliminar columnas y registros

In [31]:
# Calculamos cantidad de nulos por campo
cant_nulos_por_campo = data_caba_clean.isnull().sum()
cant_nulos_por_campo

property_type                     0
place_name                        0
price                          2959
currency                       2959
price_aprox_local_currency     2959
price_aprox_usd                2959
surface_total_in_m2            5707
surface_covered_in_m2          1963
price_usd_per_m2               8416
price_per_m2                   4570
floor                         28070
rooms                         15748
description                       0
dtype: int64

In [32]:
# Calculamos porcentaje de nulos por campo
cant_registros = data_caba_clean.shape[0]
print(f'Cantidad Original de registros: {cant_registros}\n')
#len(data.index) es lo mismo

porc_nulos_por_campo = np.round((100 * cant_nulos_por_campo / cant_registros),2)
print(porc_nulos_por_campo)

Cantidad Original de registros: 31019

property_type                  0.00
place_name                     0.00
price                          9.54
currency                       9.54
price_aprox_local_currency     9.54
price_aprox_usd                9.54
surface_total_in_m2           18.40
surface_covered_in_m2          6.33
price_usd_per_m2              27.13
price_per_m2                  14.73
floor                         90.49
rooms                         50.77
description                    0.00
dtype: float64


In [33]:
data_caba.shape

(32316, 26)

In [34]:
data_caba.head(20)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.00,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.00,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,95000.0,USD,1676227.50,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,138000.0,USD,2434941.00,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,195000.0,USD,3440677.50,195000.0,65.0,60.0,3000.000000,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
13,13,sell,apartment,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,3430234.0,NaN,NaN,NaN,111700.0,USD,1970890.65,111700.0,50.0,30.0,2234.000000,3723.333333,NaN,1.0,NaN,http://www.properati.com.ar/15bp9_venta_depart...,Torre I Mondrian. 3 ambientes con terraza y d...,Vitraux Palermo,https://thumbs4.properati.com/2/ZTUjkRzTz6YvcU...
14,14,sell,apartment,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,3430234.0,NaN,NaN,NaN,147900.0,USD,2609621.55,147900.0,42.0,31.0,3521.428571,4770.967742,NaN,1.0,NaN,http://www.properati.com.ar/15bpa_venta_depart...,Torre II Dalí. Ambiente unico divisible.Vitrau...,Vitraux Palermo,https://thumbs4.properati.com/8/Uay2dC732CNlRw...
16,16,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6523561177,-58.5016239381",-34.652356,-58.501624,239000.0,USD,4217035.50,239000.0,140.0,98.0,1707.142857,2438.775510,NaN,4.0,NaN,http://www.properati.com.ar/15bpj_venta_ph_mat...,Ventas Mataderos al frente Duplex 4 amb.- Plan...,VENTA-MATADEROS-DUPLEX 4 AMB.,https://thumbs4.properati.com/7/XSdJtiE8ZkWgon...
19,19,sell,apartment,Palermo,|Argentina|Capital Federal|Palermo|,Argentina,Capital Federal,3430234.0,"-34.580503566,-58.4058744847",-34.580504,-58.405874,350000.0,USD,6175575.00,350000.0,104.0,96.0,3365.384615,3645.833333,NaN,3.0,NaN,http://www.properati.com.ar/15bq8_venta_depart...,Excelente semipiso al contra frente en Bulnes ...,"Bulnes y Libertador: espectacular pulmón, con ...",https://thumbs4.properati.com/8/1y9fKHLee-aQQj...
21,21,sell,apartment,Pale

## De los datos filtrados según Desafio 1 deberiamos:

1. Terminar de dropear variables incompletas que serian necesarias para poder entrenar un modelo. (ejemplo: si la propiedad no posee: price, price_aprox_local_currency,price_usd_per_m2, price_per_m2, etc...) la información de los barrios ya fue analizado y dropeado en el paso anterior.
2. Definir las variables CUANTITATIVAS, ejemplo precio, total de m2 de superficie.
3. Armar las variables CUALITATIVAS, Dummy (ejemplo: place_name, property_type, amenities, rooms).
4. Armar una nueva variable Dummy que contemple los precios de propiedad segun su distribución por metro cuadrado. Utilizando Panda CUT. 

## Finalizado el analisis de datos, armamos diferentes modelos para predecir el precio de la propiedad.

1. Regresion Lineal Multiple
2. Entrenar los modelos con Lasso, RidgeCV y utilizando Cross Validation.



### Primero damos homogenidad a los datos

Vamos a realizar ciertas verificaciones de los datos.

In [35]:
# 1. Verificamos que el price y el price_aprox_usd sean los mismos, y luego si fuera así dropeamos el price_aprox_usd

mask_price_validation = data_caba_clean.price != data_caba_clean.price_aprox_usd

display(data_caba_clean.loc[mask_price_validation, :].shape)

# Detectamos que hay 4138 propiedades donde los valores son diferentes? pero estan en Pesos o son NaN?

display(data_caba_clean.loc[mask_price_validation, :].sample(10))

# Detectamos que tenemos los dos casos, donde los precios son NaN y donde la moneda está en ARS (lo cual no estaría mal)
# En el caso de ARS, vemos que el valor de price toma el de la moneda local ~ Por lo que podriamos llegar a eliminar price, currency y price_aprox_local_currency 

# Para los valores de NaN tendriamos que tomar la decisión si los dropeamos o tratamos de calcular un precio promedio en base a la cantidad de metros cuadrados. Podemos hacerlo en base al calculo que realizamos en el Desafio 1 ??

(4138, 13)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
86518,apartment,Almagro,NaN,NaN,NaN,NaN,47.0,43.0,NaN,NaN,NaN,NaN,"Tipologia ""C"": Monoambiente divisible de 47.05..."
13637,house,Villa Luro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CODIGO: 1019-V303 ubicado en: VICTOR HUGO 700 ...
8062,apartment,Constitución,1311800.0,ARS,1298115.74,73570.56,NaN,31.0,NaN,42316.129032,NaN,1.0,* Balcón al FrenteEstá ubicado en una zona est...
86300,apartment,Almagro,NaN,NaN,NaN,NaN,43.0,35.0,NaN,NaN,NaN,NaN,BBU3180 AP207670
117986,apartment,Belgrano,NaN,NaN,NaN,NaN,78.0,68.0,NaN,NaN,NaN,NaN,"3 ambientes con balcón.2 dormitorios, baño, li..."
48032,apartment,San Telmo,1329600.0,ARS,1315730.07,74568.85,30.0,26.0,2485.628333,51138.461538,NaN,1.0,Edificio en construcción En una de las avenida...
7002,house,Parque Avellaneda,NaN,NaN,NaN,NaN,250.0,220.0,NaN,NaN,NaN,NaN,CODIGO: 1434-TC-0129 ubicado en: Pringles 4682...
120771,apartment,Belgrano,NaN,NaN,NaN,NaN,65.0,57.0,NaN,NaN,NaN,NaN,2 AMBIENTES AMPLIOS CON BALCÓN...
32129,PH,Flores,NaN,NaN,NaN,NaN,65.0,55.0,NaN,NaN,NaN,3.0,"3 AMB ENTRADA INDEPENDIENTE AL FRENTE, PISO ÚN..."
67227,apartment,Palermo Soho,NaN,NaN,NaN,NaN,42.0,28.0,NaN,NaN,NaN,NaN,Ambiente moderno. Cocina con barra.Amplia terr...


In [36]:
data_caba_clean.shape

(31019, 13)

<h3> Eliminamos registros según un umbral de datos faltantes </h3>

[volver a TOC](#section_toc)

In [37]:
# Elimino primero los registros donde no tengo nada de informacion. 

umbral = 10
data_caba_clean.dropna(axis = 0, thresh=umbral, inplace=True)
# OJO que utilizando umbral de 10 se fueron casi 10000 datos incompletos. 
#--> se pierden 6000 datos aprox (ver shape de abajo), un 20% de los datos aprox

# Vuelvo a ver como quedaron los datos
mask_price_validation = data_caba_clean.price != data_caba_clean.price_aprox_usd
display(data_caba_clean.loc[mask_price_validation, :].shape)

display(data_caba_clean.shape)
display(data_caba_clean.loc[mask_price_validation, :].sample(20))


(1131, 13)

(25627, 13)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
94076,apartment,Villa Urquiza,3712600.0,ARS,3673871.62,208216.25,76.0,72.0,2739.687500,51563.888889,NaN,NaN,Lindisimo y amplio tres ambientes c a ESTRENAR...
75923,store,Once,1500000.0,ARS,1500893.04,85062.94,80.0,80.0,1063.286750,18750.000000,NaN,NaN,Fondo de Comercio Se vende en ONCE. Local de ...
44045,apartment,Recoleta,1845000.0,ARS,1825753.69,103474.38,56.0,26.0,1847.756786,70961.538462,NaN,1.0,1 AMBIENTE AL CONTRAFRENTE CON TERRAZA EXCLUSI...
13582,apartment,Once,2164648.0,ARS,2142067.17,121401.41,NaN,50.0,NaN,43292.960000,NaN,1.0,Departamento ideal para estudiantesSup. Total:...
114217,apartment,Villa Urquiza,4367385.0,ARS,4321826.18,244939.00,NaN,83.0,NaN,52619.096386,4.0,2.0,Corredor Responsable: Rebeca Stern - C.U.C.I.C...
118540,apartment,San Telmo,1634100.0,ARS,1617053.66,91646.33,42.0,39.0,2182.055476,41900.000000,NaN,NaN,"Departamento 42m² en Defensa 1657, Capital Fed..."
120386,apartment,San Telmo,1310100.0,ARS,1296433.51,73475.22,35.0,32.0,2099.292000,40940.625000,NaN,NaN,"Departamento 35m², Capital Federal, San Telmo,..."
28073,apartment,Palermo,2726783.0,ARS,2698338.27,152928.01,NaN,55.0,NaN,49577.872727,102.0,2.0,Corredor Responsable: Rebeca Stern - C.U.C.I.C...
112339,apartment,San Telmo,1308400.0,ARS,1294751.29,73379.88,34.0,31.0,2158.231765,42206.451613,NaN,NaN,Monoambiente en el barrio de San Telmo y con ...
72522,apartment,Belgrano,9435376.0,ARS,9336950.12,529170.57,167.0,101.0,3168.686048,93419.564356,NaN,NaN,BELGRANO - DUPLEX EN VENTA DE 3 AMBIENTES C/BA...


In [38]:
# Para la superficie de metros cuadadros totales, podemos creer que en caso de ser Nan 
#es porque es un departamente y la cantidad de m2 cubierta sería iguala la cantidad de m2 totales.data_caba

data_caba_clean.surface_total_in_m2.fillna(data_caba_clean.surface_covered_in_m2, inplace=True)

data_caba_clean.sample(10)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
89916,apartment,Belgrano,109715.0,USD,1935866.31,109715.00,47.0,42.0,2334.361702,2612.261905,NaN,NaN,"Departamento en venta de un ambiente, baño com..."
42078,apartment,Flores,1395453.0,ARS,1380896.15,78262.13,37.0,33.0,2115.192703,42286.454545,NaN,1.0,"Venta de Departamento 1 AMBIENTE en Flores, Ca..."
111099,apartment,Boca,79000.0,USD,1393915.50,79000.00,43.0,43.0,1837.209302,1837.209302,NaN,2.0,Departamento de dos ambientes estilo loft. Sin...
102074,house,Belgrano,395000.0,USD,6969577.50,395000.00,120.0,120.0,3291.666667,3291.666667,NaN,NaN,casa terreno venta nuñez lote propio posibilid...
15175,house,Caballito,330000.0,USD,5822685.00,330000.00,265.0,265.0,NaN,1245.283019,NaN,4.0,Corredor Responsable: Hernan Perrone - C.U.C.I...
110352,apartment,Versalles,111000.0,USD,1958539.50,111000.00,55.0,55.0,2018.181818,2018.181818,2.0,3.0,"Departamento 3 AMBIENTES, muy luminoso. Living..."
26345,apartment,Boedo,40000.0,USD,705780.00,40000.00,30.0,30.0,1333.333333,1333.333333,NaN,1.0,avb687- Invierta seguro - tu depa por $ 600.00...
35795,apartment,Barrio Norte,68500.0,USD,1208648.25,68500.00,25.0,25.0,2740.000000,2740.000000,NaN,NaN,CODIGO: 1772-delac3500 ubicado en: De la Carco...
9183,PH,Flores,85000.0,USD,1499782.50,85000.00,20.0,20.0,NaN,4250.000000,NaN,2.0,Corredor Responsable: Franco Savone - CUCICBA ...
116954,store,Palermo,650000.0,USD,11468925.00,650000.00,116.0,116.0,5603.448276,5603.448276,NaN,NaN,EN VENTA LOCAL COMERCIAL EN PALERMO + 4 COCHER...


<h3> Completamos datos faltantes de superficie </h3>

[volver a TOC](#section_toc)

In [39]:
# Idem al punto anterior pero ahora de surface_total a surface_covered

data_caba_clean.surface_covered_in_m2.fillna(data_caba_clean.surface_total_in_m2, inplace=True)

data_caba_clean.sample(10)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
37094,apartment,Saavedra,215000.0,USD,3793567.50,215000.00,66.0,66.0,NaN,3257.575758,NaN,3.0,"Venta - Departamento - Argentina, Buenos Aires..."
96600,apartment,Retiro,5472110.0,ARS,5415027.17,306896.04,69.0,59.0,4447.768696,92747.627119,NaN,NaN,Importante departamento o oficina con la posib...
21,apartment,Palermo,270500.0,USD,4772837.25,270500.00,118.0,73.0,2292.372881,3705.479452,NaN,4.0,"EXCELENTE ZONA, MULTIPLES MEDIOS DE TRANSPORTE..."
69372,apartment,Belgrano,92070.0,USD,1624529.11,92070.00,29.0,27.0,3174.827586,3410.000000,NaN,NaN,Arribeños Air | Arribeños 2500 En Construcción...
7998,apartment,Caballito,355800.0,USD,6277913.10,355800.00,130.0,88.0,2736.923077,4043.181818,NaN,4.0,"Emprendimiento ícono en la zona, Caballito Nor..."
73303,apartment,Nuñez,180792.0,USD,3189984.44,180792.00,67.0,53.0,2698.388060,3411.169811,NaN,NaN,Con inmejorable ubicación sobre Avenida del Li...
69922,apartment,Nuñez,2250000.0,USD,39700125.00,2250000.00,320.0,290.0,7031.250000,7758.620690,NaN,NaN,"Torre Chateau Libertador, 4 dormitorios en sui..."
111513,PH,Villa Luro,165000.0,USD,2911342.50,165000.00,117.0,80.0,1410.256410,2062.500000,NaN,4.0,Dueño Vende! EXCELENTE PH - 4 amb con terraza ...
40793,apartment,Monserrat,57500.0,USD,1014558.75,57500.00,31.0,31.0,NaN,1854.838710,NaN,1.0,Corredor Responsable: Alejandro Botta - CUCICB...
22807,apartment,Belgrano,199500.0,USD,3520077.75,199500.00,44.0,43.0,4534.090909,4639.534884,NaN,2.0,"Excelente torre de categoría, 2 ambientes a es..."


<h3> Completamos datos faltantes de price_usd_per_m2 </h3>

[volver a TOC](#section_toc)

In [40]:
# Idem al punto anterior pero ahora de surface_total a surface_covered

data_caba_clean.price_usd_per_m2.fillna(data_caba_clean.price_aprox_usd / data_caba_clean.surface_total_in_m2, inplace=True)

data_caba_clean.sample(10)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description
17162,apartment,Constitución,93000.0,USD,1640938.50,93000.0,47.0,41.0,1978.723404,2268.292683,NaN,1.0,"EDIFICIO FACULTAD VIII, a construirse en la ca..."
110378,apartment,Villa Devoto,120000.0,USD,2117340.00,120000.0,46.0,38.0,2608.695652,3157.894737,NaN,2.0,Venta de Departamento 2 AMBIENTES en Villa Dev...
45370,apartment,San Telmo,290000.0,USD,5116905.00,290000.0,178.0,178.0,1629.213483,1629.213483,NaN,4.0,"Departamento con hall de distribución, piso de..."
60119,apartment,Belgrano,635000.0,USD,11204257.50,635000.0,179.0,171.0,3547.486034,3713.450292,NaN,3.0,EXCELENTE PISO EN AV. DEL LIBERTADOR IMPECABLE...
31894,apartment,Caballito,315000.0,USD,5558017.50,315000.0,138.0,98.0,2282.608696,3214.285714,1.0,4.0,Venta de Departamento 4 AMBIENTES en Caballito...
104660,apartment,Liniers,200000.0,USD,3528900.00,200000.0,95.0,80.0,2105.263158,2500.000000,NaN,NaN,Local Comercial en Planta Baja de 100m2 con Ba...
43031,house,Nuñez,1300000.0,USD,22937850.00,1300000.0,537.0,435.0,2420.856611,2988.505747,NaN,NaN,Increíble casa en Nuñez. Es un inmueble reside...
92843,apartment,Caballito,119120.0,USD,2101812.84,119120.0,44.0,37.0,2707.272727,3219.459459,NaN,NaN,Situado en un único lote de más de 3.900 m2 de...
111809,apartment,Monserrat,94100.0,USD,1660347.45,94100.0,47.0,41.0,2002.127660,2295.121951,NaN,NaN,Departamentos en venta con piso flotante mela...
57814,apartment,San Cristobal,108000.0,USD,1905606.00,108000.0,48.0,48.0,2250.000000,2250.000000,12.0,2.0,Corredor Responsable: Martin Macagno - CUCICBA...


<h3> Eliminamos floor </h3>

[volver a TOC](#section_toc)

In [41]:
display(data_caba.floor.isnull().count())

# El piso es un dato muy incompleto por lo que se dropea

# Dropeamos la columna creada anteriormente.
data_caba_clean.drop(columns=['floor'], inplace=True)

# En el caso de Rooms lo reemplazamos por 0 --> Lo pasé más abajo
# data_caba_clean.rooms.fillna(0, inplace=True)

32316

In [42]:
display(data_caba.rooms.count())
# En el caso de rooms, podemos tomar que el dato para los NaN es 0?? modifica esto el analisis para predecir? 

15325

<h3> Eliminamos columnas de precios en moneda local </h3>

[volver a TOC](#section_toc)

In [43]:
# Dropeamos los valores que sabemos que está repetidos o con otra moneda.data_caba

data_caba_clean.drop(columns=['price', 'currency','price_per_m2'], inplace=True)

data_caba_clean.sample(20)

,property_type,place_name,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,rooms,description
49025,apartment,Almagro,1764450.00,100000.00,35.0,35.0,2857.142857,1.0,Corredor Responsable: Jorge Salafia - CUCICBA ...
91647,apartment,Palermo Hollywood,3123076.50,177000.00,41.0,36.0,4317.073171,NaN,VENTA DEPARTAMENTO 1 AMBIENTE PALERMO PALERMO ...
53504,apartment,San Telmo,578245.55,32772.00,35.0,24.0,936.342857,1.0,"Lanzamiento en pozo, lugar estrategico de San ..."
35919,apartment,Caballito,2981920.50,169000.00,76.0,76.0,2223.684211,3.0,Corredor Responsable: Cristian Arnal Ponti - C...
61570,apartment,Flores,1464493.50,83000.00,37.0,37.0,2243.243243,2.0,EDIFICIO PAVILLON SAN JOSE COMENZADAEXCELENTE...
11898,apartment,Palermo,2170273.50,123000.00,42.0,42.0,2928.571429,NaN,VENTA DEPARTAMENTO 2 AMBIENTES PALERMOImpecabl...
73402,apartment,Puerto Madero,13233375.00,750000.00,132.0,128.0,5681.818182,NaN,EN VENTA CON RENTAExcelente departamento amueb...
77728,apartment,Villa Luro,1235115.00,70000.00,34.0,30.0,2058.823529,NaN,"Departamentos a ESTRENAR, en Torre a entregar..."
41367,house,Palermo Hollywood,14115600.00,800000.00,487.0,487.0,1642.710472,NaN,VENTA CASA 7 AMBIENTES O MAS EN PALERMO HOLLYW...
60867,apartment,Almagro,3158365.50,179000.00,92.0,72.0,1945.652174,2.0,EDIFICIO MARAL 56 . Edificio de categoria. Exc...


<h3> Nuevo recuento de nulos </h3>

In [44]:
print('\nPorcentaje de valores incompletos por columna:')
print(f'{round(100 * data_caba_clean.isnull().sum()/data_caba_clean.shape[0], 2)}')

display(data_caba_clean.shape)


Porcentaje de valores incompletos por columna:
property_type                  0.00
place_name                     0.00
price_aprox_local_currency     0.00
price_aprox_usd                0.00
surface_total_in_m2            0.00
surface_covered_in_m2          0.00
price_usd_per_m2               0.00
rooms                         43.82
description                    0.00
dtype: float64


(25627, 9)

<h3>Creación de una columna para categorizar por metros cuadrados

In [45]:
# Delimitamos los bins para realizar un cut
# bins = [20, 30, 45, 75, 150, 220]
bins = [0, 30, 45, 75, 150, 220]
labels = ['mono', 's45', 's75', 's150', 'm220' ]

data_caba_clean['m2_categories'] = pd.cut(data_caba_clean.surface_covered_in_m2, bins, labels)
data_caba_clean['m2_labels'] = pd.cut(x=data_caba_clean.surface_covered_in_m2, bins=bins, labels=labels, right=False)


In [46]:
data_caba_clean.m2_categories.value_counts().sort_index()

(0, 30]       2530
(30, 45]      7452
(45, 75]      7227
(75, 150]     5449
(150, 220]    1572
Name: m2_categories, dtype: int64

In [47]:
data_caba_clean.m2_labels.value_counts().sort_index()

mono    1633
s45     7863
s75     7475
s150    5574
m220    1628
Name: m2_labels, dtype: int64

<a id="section_caba_description"></a> 
<h3> Análisis del campo <strong>description</strong></h3>

[volver a TOC](#section_toc)

Normalización del campo, todo minuscula, sin tildes, etc

In [48]:
data_caba_clean.description = data_caba_clean.description.str.lower()

data_caba_clean.description.sample(10)

21027     corredor responsable: marcelo trujillo - cucic...
22032     corredor responsable: alejandro botta - cucicb...
61089     venta de departamento 3 ambientes en caballito...
17556     departamento de 2 amb con balcon. cuenta de un...
23101     datos propiedad:departamento de 3 ambientes co...
33422     corredor responsable: patricia maria sodor - c...
52112     corredor responsable: veronica grela reina - c...
97572     muy buen dpto 4 amb con dependencia de servici...
105035    tercer piso contrefrente.tipo studioplanta baj...
56161     venta de departamento 3 ambientes al frente ub...
Name: description, dtype: object

<a id="section_caba_description_amb"></a> 
<h4> Cantidad de ambientes</h4>

[volver a TOC](#section_toc)

In [49]:
amb_pattern = "((?P<numero_amb>\d)(\s)?(amb|anv|amv|anb))"
amb_pattern_regex = re.compile(amb_pattern)

In [50]:
amb_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      amb_pattern_regex.search(x))
mask_amb_match_notnull = amb_match.notnull()

# Aplicamos el grupo que encuentra el dígito de cantidad de ambientes, casteado como integer
data_caba_clean.loc[mask_amb_match_notnull, 'number_rooms'] = \
amb_match.loc[mask_amb_match_notnull].apply(lambda x: int(x.group("numero_amb")))

In [51]:
dorm_pattern = "((?P<numero_dorm>\d)(\s)?(dorm))"
dorm_pattern_regex = re.compile(dorm_pattern)

In [52]:
dorm_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else dorm_pattern_regex.search(x))

mask_dorm_match_notnull = dorm_match.notnull()

data_caba_clean.loc[mask_dorm_match_notnull, 'number_rooms'] = \
dorm_match.loc[mask_dorm_match_notnull].apply(lambda x: int(x.group("numero_dorm"))+1)

#se asume que el numero de ambientes es numero_dorm +1

Tratamos de llenar los rooms que faltan con lo que obtuvimos de la descripción

In [53]:
data_caba_clean.head(3)

,property_type,place_name,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,rooms,description,m2_categories,m2_labels,number_rooms
0,PH,Mataderos,1093959.0,62000.0,55.0,40.0,1127.272727,NaN,"2 ambientes tipo casa planta baja por pasillo,...","(30, 45]",s45,2.0
2,apartment,Mataderos,1270404.0,72000.0,55.0,55.0,1309.090909,NaN,2 ambientes 3er piso lateral living comedor am...,"(45, 75]",s75,2.0
7,apartment,Belgrano,2434941.0,138000.0,45.0,40.0,3066.666667,NaN,excelente monoambiente a estrenar amplio super...,"(30, 45]",s45,NaN


In [54]:
data_caba_clean.rooms.isnull().sum()

11230

In [55]:
data_caba_clean.number_rooms.isnull().sum()

9881

In [56]:
data_caba_clean.rooms.fillna(data_caba_clean.number_rooms, inplace=True)

In [57]:
data_caba_clean.rooms.isnull().sum()

4665

In [58]:
# data_caba_clean.rooms.fillna(0, inplace=True)
# completar con los datos de categorias

In [59]:
data_caba_clean.rooms.value_counts()

2.0     5406
3.0     5262
1.0     4341
4.0     4012
5.0     1266
6.0      362
7.0      174
8.0       57
9.0       27
10.0      24
12.0      11
11.0       9
0.0        4
17.0       3
22.0       1
15.0       1
14.0       1
13.0       1
Name: rooms, dtype: int64

In [153]:
rooms_dict = {
"mono": 1, "s45":  2, "s75":  3, "s150":  4, "m220": 5}

data_caba_clean["rooms_upon_categories"] = data_caba_clean.m2_labels.map(rooms_dict)
data_caba_clean['rooms_upon_categories']

data_caba_clean.rooms.fillna(data_caba_clean.rooms_upon_categories, inplace=True)
data_caba_clean.rooms.isnull().sum()


32

### Llenamos los que faltan según las categorías de los bins???
Además: eliminar los que tengan más de 10 ambientes? Son pocos datos! 

<a id="section_caba_description_amb"></a> 
<h4> Amenities</h4>

[volver a TOC](#section_toc)

<h5> Pileta </h5>

In [154]:
pileta_pattern = "(?P<pileta>pileta|piscina|picina|pisina|pool)"
pileta_pattern_regex = re.compile(pileta_pattern)

In [155]:
pileta_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      pileta_pattern_regex.search(x))
mask_pileta_match_notnull = pileta_match.notnull()

data_caba_clean.loc[mask_pileta_match_notnull, 'pool'] = 1

In [156]:
# data_caba_clean['pool']

In [157]:
data_caba_clean['pool'].notnull().sum()

22451

In [158]:
#Completamos los datos NaN con ceros (convertimos en una variable dummy)
data_caba_clean.pool.fillna(0, inplace = True)

<h5> Laundry </h5>

In [159]:
laudry_pattern = "(?P<laundry>laundry|lavadero)"
laundry_pattern_regex = re.compile(laudry_pattern)

laundry_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      laundry_pattern_regex.search(x))
mask_laundry_match_notnull = laundry_match.notnull()

data_caba_clean.loc[mask_laundry_match_notnull, 'laundry'] = 1

In [160]:
data_caba_clean['laundry'].notnull().sum()

22451

In [161]:
data_caba_clean.laundry.fillna(0, inplace = True)

<h5> Parking </h5>

In [162]:
parking_pattern = "(?P<parking>parking|estacionamiento|garage|cochera|garaje)"
parking_pattern_regex = re.compile(parking_pattern)

parking_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      parking_pattern_regex.search(x))
mask_parking_match_notnull = parking_match.notnull()

data_caba_clean.loc[mask_parking_match_notnull, 'parking'] = 1

In [163]:
data_caba_clean['parking'].notnull().sum()

22451

In [164]:
data_caba_clean.parking.fillna(0, inplace = True)

<h5> Gimnasio </h5>

In [165]:
gym_pattern = "(?P<gimnasio>gim|gym|fitness|fitnes|ejercicio)"
gym_pattern_regex = re.compile(gym_pattern)

gym_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      gym_pattern_regex.search(x))
mask_gim_match_notnull = gym_match.notnull()

data_caba_clean.loc[mask_gim_match_notnull, 'gym'] = 1

In [166]:
data_caba_clean['gym'].notnull().sum()

22451

In [167]:
data_caba_clean.gym.fillna(0, inplace = True)

In [ ]:
plt.figure(figsize=(13,10))
sns.heatmap(data_caba_clean.corr(),annot=True)
plt.title("Correlation Heatmap")
plt.show()

<a id="section_caba_dummies"></a> 
<h3> Creación dummies sobre datos categóricos </h3>

[volver a TOC](#section_toc)

In [ ]:
# Usamos la función get_dummies con one-hot encoding (drop_first=True)
property_type_dummies = pd.get_dummies(data_caba_clean['property_type'], drop_first = True, prefix='prop_type')
property_type_dummies

In [ ]:
m2_labels_dummies = pd.get_dummies(data_caba_clean['m2_labels'], drop_first = True, prefix='m2_label')

In [ ]:
place_name_dummies = pd.get_dummies(data_caba_clean['place_name'], drop_first = True, prefix='place_name')

In [ ]:
# Hacemos un join para concatenar las columnas al dataframe
data_caba_clean = data_caba_clean.join([property_type_dummies,m2_labels_dummies, place_name_dummies])

In [ ]:
data_caba_clean.head(3)

### Falta reindexar las filas.

### Falta el dummy de Rooms pero primero hay que ver si completamos los datos con la info de los bins

In [ ]:
# Exportamos el archivo para usar en otras notebooks, esto lo guarda en el mismo directorio en el que estamos operando
# data_caba_clean.to_csv('data_caba_clean.csv')

1. Primer modelo
Eliminar rooms nulos
Modelar 

2. Imputar datos faltantes en los rooms
modelar 

3. Eliminar outliers
modelar 

In [ ]:
# Funcion para realizar el entrenamiento pero con un set de testeo.

from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn import metrics, linear_model


def train_test_error(feature_cols, target):
    X = data_caba_clean[feature_cols]
    y = data_caba_clean[target]
    # OJO que no trabajamos con tiempo por lo tanto debe ser shuffle=True.
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.65,random_state=10)
    
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    
    print (feature_cols)
    print ("y_test sample: ",y_test.values[0:10])
#     print ("y_test sample: ",y_test)
    print ("y_pred sample: ",y_pred[0:10].astype(int))
    print ('MAE:', metrics.mean_absolute_error(y_test, y_pred))
    print ('MSE:', metrics.mean_squared_error(y_test, y_pred))
    print ('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print ('R2:', metrics.r2_score(y_test, y_pred))
    print ()
    return 

def train_test_error_lazzo(feature_cols, target):
    X = data_caba_clean[feature_cols]
    y = data_caba_clean[target]
#     cv = KFold(5, shuffle=True)
    cv = 3
    # OJO que no trabajamos con tiempo por lo tanto debe ser shuffle=True.
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, random_state=10)
    
    model = linear_model.LassoCV().fit(X_train, y_train)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
    print(dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean(), zero_coefs=(model.coef_ == 0).sum()))
    print ("r^2:", model.score(X, y))
    print (model.alpha_)
    return 

In [ ]:
data_caba_clean.columns

In [ ]:
# Regresion lineal Simple

feature_cols = ['surface_total_in_m2']
target = ['price_usd_per_m2']

train_test_error(feature_cols, target)

# Incluir grafico.!!

sns.lmplot(x="surface_total_in_m2", y="price_usd_per_m2", data=data_caba_clean)
plt.title("? ")
plt.show()


In [ ]:
feature_cols = ['surface_total_in_m2', 'place_name_Agronomía', 'place_name_Almagro', 'place_name_Balvanera', 'place_name_Barracas',
                'place_name_Barrio Norte', 'place_name_Belgrano', 'place_name_Boca', 'place_name_Boedo', 'place_name_Caballito',
       'place_name_Catalinas', 'place_name_Centro / Microcentro',
       'place_name_Chacarita', 'place_name_Coghlan', 'place_name_Colegiales',
       'place_name_Congreso', 'place_name_Constitución', 'place_name_Flores',
       'place_name_Floresta', 'place_name_Las Cañitas', 'place_name_Liniers',
       'place_name_Mataderos', 'place_name_Monserrat',
       'place_name_Monte Castro', 'place_name_Nuñez', 'place_name_Once',
       'place_name_Palermo', 'place_name_Palermo Chico',
       'place_name_Palermo Hollywood', 'place_name_Palermo Soho',
       'place_name_Palermo Viejo', 'place_name_Parque Avellaneda',
       'place_name_Parque Centenario', 'place_name_Parque Chacabuco',
       'place_name_Parque Chas', 'place_name_Parque Patricios',
       'place_name_Paternal', 'place_name_Pompeya', 'place_name_Puerto Madero',
       'place_name_Recoleta', 'place_name_Retiro', 'place_name_Saavedra',
       'place_name_San Cristobal', 'place_name_San Nicolás',
       'place_name_San Telmo', 'place_name_Tribunales',
       'place_name_Velez Sarsfield', 'place_name_Versalles',
       'place_name_Villa Crespo', 'place_name_Villa Devoto',
       'place_name_Villa General Mitre', 'place_name_Villa Lugano',
       'place_name_Villa Luro', 'place_name_Villa Ortuzar',
       'place_name_Villa Pueyrredón', 'place_name_Villa Real',
       'place_name_Villa Riachuelo', 'place_name_Villa Santa Rita',
       'place_name_Villa Soldati', 'place_name_Villa Urquiza',
       'place_name_Villa del Parque']
target = ['price_usd_per_m2']

train_test_error(feature_cols, target)

# Incluir grafico.!!

sns.lmplot(x="surface_total_in_m2", y="price_usd_per_m2", data=data_caba_clean)
plt.title("? ")
plt.show()

In [ ]:
feature_cols = ['surface_total_in_m2', 'price_aprox_local_currency', 'price_aprox_usd', 'surface_covered_in_m2']
target = ['price_usd_per_m2']

train_test_error(feature_cols, target)

# Incluir grafico.!!

sns.lmplot(x="surface_total_in_m2", y="price_usd_per_m2", data=data_caba_clean)
plt.title("? ")
plt.show()

In [ ]:
feature_cols = ['price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
                'pool', 'laundry', 'parking', 'gym']
target = ['price_usd_per_m2']

train_test_error(feature_cols, target)

# Incluir grafico.!!

sns.lmplot(x="surface_total_in_m2", y="price_usd_per_m2", data=data_caba_clean)
plt.title("? ")
plt.show()

## Recordar que hay variables ROOM es INT!!!!

In [ ]:
feature_cols = ['surface_total_in_m2']
target = ['price_usd_per_m2']

train_test_error_lazzo(feature_cols, target)

# Incluir grafico.!!

sns.lmplot(x="surface_total_in_m2", y="price_usd_per_m2", data=data_caba_clean)
plt.title("? ")
plt.show()

In [ ]:
feature_cols = ['surface_total_in_m2', 'place_name_Agronomía', 'place_name_Almagro', 'place_name_Balvanera', 'place_name_Barracas',
                'place_name_Barrio Norte', 'place_name_Belgrano', 'place_name_Boca', 'place_name_Boedo', 'place_name_Caballito',
       'place_name_Catalinas', 'place_name_Centro / Microcentro',
       'place_name_Chacarita', 'place_name_Coghlan', 'place_name_Colegiales',
       'place_name_Congreso', 'place_name_Constitución', 'place_name_Flores',
       'place_name_Floresta', 'place_name_Las Cañitas', 'place_name_Liniers',
       'place_name_Mataderos', 'place_name_Monserrat',
       'place_name_Monte Castro', 'place_name_Nuñez', 'place_name_Once',
       'place_name_Palermo', 'place_name_Palermo Chico',
       'place_name_Palermo Hollywood', 'place_name_Palermo Soho',
       'place_name_Palermo Viejo', 'place_name_Parque Avellaneda',
       'place_name_Parque Centenario', 'place_name_Parque Chacabuco',
       'place_name_Parque Chas', 'place_name_Parque Patricios',
       'place_name_Paternal', 'place_name_Pompeya', 'place_name_Puerto Madero',
       'place_name_Recoleta', 'place_name_Retiro', 'place_name_Saavedra',
       'place_name_San Cristobal', 'place_name_San Nicolás',
       'place_name_San Telmo', 'place_name_Tribunales',
       'place_name_Velez Sarsfield', 'place_name_Versalles',
       'place_name_Villa Crespo', 'place_name_Villa Devoto',
       'place_name_Villa General Mitre', 'place_name_Villa Lugano',
       'place_name_Villa Luro', 'place_name_Villa Ortuzar',
       'place_name_Villa Pueyrredón', 'place_name_Villa Real',
       'place_name_Villa Riachuelo', 'place_name_Villa Santa Rita',
       'place_name_Villa Soldati', 'place_name_Villa Urquiza',
       'place_name_Villa del Parque', 'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
                'pool', 'laundry', 'parking', 'gym']
target = ['price_usd_per_m2']

train_test_error_lazzo(feature_cols, target)

# Incluir grafico.!!


# Necesitamos definir Outliers.

In [ ]:
fig, ax = plt.subplots()
bp = ax.boxplot(data_caba_clean['surface_covered_in_m2'].dropna(), vert=False, showmeans=True)


In [ ]:
# Definimos outliers aproximados
lower_bound = 0.01
upper_bound = 0.95
rest = data_caba_clean.price_usd_per_m2.quantile([lower_bound, upper_bound])
rest

In [ ]:
# Creamos una mask entendiendo que los precios minimos y maximos pueden rondar los 600 USD y 4000 USD y fijamos outliers aproximados

precioxm2_mask = (data_caba_clean.price_usd_per_m2 >= rest[0.01]) & (data_caba_clean.price_usd_per_m2 <= rest[0.95])
data_caba_clean = data_caba_clean.loc[precioxm2_mask, :]

# Vuelvo a graficar.

plt.hist(data_caba_clean.price_usd_per_m2, bins=20, rwidth=0.8, color = '#29788E')
plt.xlabel = 'Precio x M2'
plt.ylabel = 'Count'
plt.show()

print(data_caba_clean.price_usd_per_m2.max()) 
print(data_caba_clean.price_usd_per_m2.min())

# Parece una distribución normal? sino fuera por los valores elevados que tenemos a partir de los 3500...

In [ ]:
# Definimos outliers aproximados
lower_bound = 0.001
upper_bound = 0.99
rest = data_caba_clean.surface_covered_in_m2.quantile([lower_bound, upper_bound])
rest

In [ ]:
# Creamos una mask entendiendo que los precios minimos y maximos pueden rondar los 600 USD y 4000 USD y fijamos outliers aproximados

surface_xm2_mask = (data_caba_clean.surface_covered_in_m2 >= rest[0.001]) & (data_caba_clean.surface_covered_in_m2 <= rest[0.99])
data_caba_clean = data_caba_clean.loc[surface_xm2_mask, :]

# Vuelvo a graficar.

plt.hist(data_caba_clean.surface_covered_in_m2, bins=20, rwidth=0.8, color = '#29788E')
plt.xlabel = 'Precio x M2'
plt.ylabel = 'Count'
plt.show()

print(data_caba_clean.surface_covered_in_m2.max()) 
print(data_caba_clean.surface_covered_in_m2.min())

# Parece una distribución normal? sino fuera por los valores elevados que tenemos a partir de los 3500...

In [ ]:
feature_cols = ['surface_total_in_m2','prop_type_apartment', 'prop_type_house', 'prop_type_store', 'place_name_Agronomía', 'place_name_Almagro', 'place_name_Balvanera', 'place_name_Barracas',
                'place_name_Barrio Norte', 'place_name_Belgrano', 'place_name_Boca', 'place_name_Boedo', 'place_name_Caballito',
       'place_name_Catalinas', 'place_name_Centro / Microcentro',
       'place_name_Chacarita', 'place_name_Coghlan', 'place_name_Colegiales',
       'place_name_Congreso', 'place_name_Constitución', 'place_name_Flores',
       'place_name_Floresta', 'place_name_Las Cañitas', 'place_name_Liniers',
       'place_name_Mataderos', 'place_name_Monserrat',
       'place_name_Monte Castro', 'place_name_Nuñez', 'place_name_Once',
       'place_name_Palermo', 'place_name_Palermo Chico',
       'place_name_Palermo Hollywood', 'place_name_Palermo Soho',
       'place_name_Palermo Viejo', 'place_name_Parque Avellaneda',
       'place_name_Parque Centenario', 'place_name_Parque Chacabuco',
       'place_name_Parque Chas', 'place_name_Parque Patricios',
       'place_name_Paternal', 'place_name_Pompeya', 'place_name_Puerto Madero',
       'place_name_Recoleta', 'place_name_Retiro', 'place_name_Saavedra',
       'place_name_San Cristobal', 'place_name_San Nicolás',
       'place_name_San Telmo', 'place_name_Tribunales',
       'place_name_Velez Sarsfield', 'place_name_Versalles',
       'place_name_Villa Crespo', 'place_name_Villa Devoto',
       'place_name_Villa General Mitre', 'place_name_Villa Lugano',
       'place_name_Villa Luro', 'place_name_Villa Ortuzar',
       'place_name_Villa Pueyrredón', 'place_name_Villa Real',
       'place_name_Villa Riachuelo', 'place_name_Villa Santa Rita',
       'place_name_Villa Soldati', 'place_name_Villa Urquiza',
       'place_name_Villa del Parque', 'price_aprox_usd',
                'pool', 'laundry', 'parking', 'gym']
target = ['price_usd_per_m2']

train_test_error_lazzo(feature_cols, target)
train_test_error(feature_cols, target)